In [1]:
# Importing the required Libraries
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import glob
import os
import shutil
import nltk
from nltk.tokenize import RegexpTokenizer, sent_tokenize
import re
#nltk.download('punkt') #Run only once in a machine

In [2]:
# Reading the Input file
os.chdir(r'C:\Users\krish\DATA\Blackcoffer_Assignment_Project') # Path - Input Excel file
data = pd.read_csv("Input.csv")

In [3]:
# Viewing the dataframe
data.head()

,URL_ID,URL
0,37,https://insights.blackcoffer.com/ai-in-healthc...
1,38,https://insights.blackcoffer.com/what-if-the-c...
2,39,https://insights.blackcoffer.com/what-jobs-wil...
3,40,https://insights.blackcoffer.com/will-machine-...
4,41,https://insights.blackcoffer.com/will-ai-repla...


In [4]:
# Converting the column into list
url = data["URL"].tolist()
url_id = data["URL_ID"].tolist()

In [5]:
# Requesting the data from the URL (Website) using requests module and beautifulsoup module to take out only the text that is required
os.chdir(r'C:\Users\krish\DATA\Blackcoffer_Assignment_Project\Text_Files') # Path - A folder where all the text files with URL_ID

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
for i in range(len(url)):
    # Creating the file name to save each URL's data into a text file
    filename = str(url_id[i]) + '.txt'
    r = requests.get(url[i],headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    top = soup.title
    top = top.string
    top = top.removesuffix("- Blackcoffer Insights")
    bot = ""
    elements = soup.find_all('p') 
    for element in elements:
        bot = bot + element.text
    file = open(filename,'w',encoding="utf-8")
    file.write(top + bot)
    file.close()

In [6]:
# Combining and reading all the stopwords file from the folder and deleting the output file at end
os.chdir(r'C:\Users\krish\DATA\Blackcoffer_Assignment_Project\StopWords') # Path - The folder which contains all the StopWords
my_files = glob.glob('*.txt') 
 
with open('output_file.txt','wb') as wfd: 
    for f in my_files: 
        with open(f,'rb') as fd: 
            shutil.copyfileobj(fd, wfd)
file.close()

file = open('output_file.txt','r')
dat = file.read()
file.close()
dat = dat.split( )

os.remove("output_file.txt")

In [7]:
# Combining all the URL's data into a single list
text_op = []
full_para_list = []
os.chdir(r'C:\Users\krish\DATA\Blackcoffer_Assignment_Project\Text_Files') # Path - The folder which has all the IRL_ID text files
my_op = glob.glob('*.txt')
    
for i in my_op:
    file = open(i,'r',encoding="utf8")
    full_para = file.read()
    file.close()
    
    full_para_list.append(full_para)
    para = full_para.split()
    resultwords  = [word for word in para if word.lower() not in dat]
    result = ' '.join(resultwords)
    text_op.append(result)

In [8]:
# Tokenizing each text file's data and forming a combined list
final = []
tokenizer = RegexpTokenizer('\w+')
stopword_token = tokenizer.tokenize(str(dat))

tempo = []
for i in text_op:
    words_token = tokenizer.tokenize(str(i))
    tempo.append(words_token)   

In [9]:
# Removing all the words in text files that are in the stopword list
final = []
count = 0
for i in tempo:
    words = []
    for j in i:
        if j not in dat:
            words.append(j)
    final.append(words)

In [10]:
# Reading the positive and negative words into a list
os.chdir(r'C:\Users\krish\DATA\Blackcoffer_Assignment_Project\MasterDictionary') # Path - The folder which has positive and negative words file

file = open('positive-words.txt','r')
positive_words = file.read()
file.close()
positive_words = positive_words.split( )

file = open('negative-words.txt','r')
negative_words = file.read()
file.close()
negative_words = negative_words.split( )

In [11]:
# Counting the no of positive and negative words in each text file
posi_list = []
neg_list = []
for i in final:
    positive_len = 0
    for j in i: 
        if j in positive_words:
            positive_len = positive_len+1
    posi_list.append(positive_len)
    
    negative_len = 0
    for j in i: 
        if j in negative_words:
            negative_len = negative_len+1
    neg_list.append(negative_len)

In [12]:
# Finding total no of text in the text files
no_of_words = []
for i in tempo:
    no_of_words.append(len(i))

In [13]:
# Finding total no of sentence in the text files
no_of_sentence= []
for i in full_para_list:
    number_of_sentences = sent_tokenize(i)
    no_of_sentence.append(len(number_of_sentences))

In [14]:
# Finding the number of complex words in the text files
def syllable(word):
    word = word.lower()
    count = 0
    vowels = "aeiouy"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
    if word.endswith("e"):
        count -= 1
    if count == 0:
        count += 1
    return count

no_of_complex_words_list = []
syllable_list = []
for w in tempo:
    no_of_complex_words = 0
    syllable_word_list = []
    for x in w:
        syllable_count = syllable(x)
        if syllable_count > 1:
            no_of_complex_words = no_of_complex_words + 1
        syllable_word_list.append(syllable_count)
    syllable_list.append(np.mean(syllable_word_list))
    no_of_complex_words_list.append(no_of_complex_words)

In [15]:
# Finding the personal pronouns in the text files
personal_pronouns = []
for i in full_para_list:
    pronounRegex = re.compile(r'\b(I|we|my|ours|(?-i:us))\b',re.I)
    pronouns = pronounRegex.findall(i)
    personal_pronouns.append(len(pronouns))

In [16]:
# Total number of character count in the text files
char_count = []
for i in full_para_list:
    count = 0
    for j in i: 
        for k in j:
            if(k != ' '):
                count = count + 1
    char_count.append(count)

In [17]:
# Combining all the gathered data into the dataframe
data['POSITIVE_SCORE'] = posi_list
data['NEGATIVE_SCORE'] = neg_list
#Polarity Score = (Positive Score – Negative Score)/ ((Positive Score + Negative Score) + 0.000001)
data['POLARITY_SCORE'] = (data["POSITIVE_SCORE"] - data["NEGATIVE_SCORE"]) / ((data["POSITIVE_SCORE"] + data["NEGATIVE_SCORE"]) + 0.000001)
#Subjectivity Score = (Positive Score + Negative Score)/ ((Total Words after cleaning) + 0.000001)
data['No_of_words'] = no_of_words
data['SUBJECTIVITY_SCORE'] = (data["POSITIVE_SCORE"] + data["NEGATIVE_SCORE"]) / ((data['No_of_words']) + 0.000001)
#Average Sentence Length = the number of words / the number of sentences
data['No_of_sentence'] =  no_of_sentence
data['AVG_SENTENCE_LENGTH'] = data["No_of_words"] / data["No_of_sentence"]
data['No_of_complex_words'] = no_of_complex_words_list
data['PERCENTAGE_OF_COMPLEX_WORDS'] = data["No_of_complex_words"] / data["No_of_words"]
#Fog Index = 0.4 * (Average Sentence Length + Percentage of Complex words)
data['FOG_INDEX'] = (data['AVG_SENTENCE_LENGTH'] + data['PERCENTAGE_OF_COMPLEX_WORDS']) * 0.4
#Average Number of Words Per Sentence = the total number of words / the total number of sentences
data['AVG_NUMBER_OF_WORDS_PER_SENTENCE'] = data['No_of_words'] / data['No_of_sentence']
data['COMPLEX_WORD_COUNT'] = data['No_of_complex_words']
data['WORD_COUNT'] = data['No_of_words']
data['SYLLABLE_PER_WORD'] = syllable_list
data['PERSONAL_PRONOUNS'] = personal_pronouns
#Sum of the total number of characters in each word/Total number of words
data['char_count'] = char_count
data['AVG_WORD_LENGTH'] = data['char_count'] / data['WORD_COUNT']

data = data.drop(['No_of_words','No_of_sentence', 'No_of_complex_words', 'char_count'], axis=1)

In [18]:
data.tail()

,URL_ID,URL,POSITIVE_SCORE,NEGATIVE_SCORE,POLARITY_SCORE,SUBJECTIVITY_SCORE,AVG_SENTENCE_LENGTH,PERCENTAGE_OF_COMPLEX_WORDS,FOG_INDEX,AVG_NUMBER_OF_WORDS_PER_SENTENCE,COMPLEX_WORD_COUNT,WORD_COUNT,SYLLABLE_PER_WORD,PERSONAL_PRONOUNS,AVG_WORD_LENGTH
109,146,https://insights.blackcoffer.com/blockchain-fo...,28,30,-0.034483,0.099828,16.138889,0.650602,6.715797,16.138889,378,581,2.067126,3,9.327022
110,147,https://insights.blackcoffer.com/the-future-of...,16,41,-0.438596,0.055072,23.522727,0.683092,9.682328,23.522727,707,1035,2.193237,1,8.956522
111,148,https://insights.blackcoffer.com/big-data-anal...,16,43,-0.457627,0.111321,16.060606,0.671698,6.692922,16.060606,356,530,2.147170,5,9.335849
112,149,https://insights.blackcoffer.com/business-anal...,34,45,-0.139241,0.134354,8.647059,0.654762,3.720728,8.647059,385,588,2.112245,37,10.132653
113,150,https://insights.blackcoffer.com/challenges-an...,22,26,-0.083333,0.087751,18.233333,0.667276,7.560244,18.233333,365,547,2.199269,12,10.118830


In [19]:
# Writing the DataFrame into a Excel file
os.chdir(r'C:\Users\krish\DATA\Blackcoffer_Assignment_Project') # Path - The output file will be stored
data.to_excel("Output Data Structure.xlsx",index=False)